# Simplicity

The safe language for smart contracts on bitcoin.

In [24]:
#program;;
module Fmt = CCFormat;;
#logic;;

module Fmt = CCFormat


## Basic types

In [25]:
type ty =
  | One
  | Prod of ty * ty
  | Sum of ty * ty
  
let one = One
let prod a b = Prod(a,b)
let sum a b = Sum (a,b)


(** a sequent, i.e. a pair of and input type and an output type *)
type top_ty = ty * ty


type ty = One | Prod of ty * ty | Sum of ty * ty
val one : ty = One
val prod : ty -> ty -> ty = <fun>
val sum : ty -> ty -> ty = <fun>
type top_ty = ty * ty


In [26]:
#program;;

let rec pp_ty out = function
  | One -> Fmt.string out "1"
  | Prod (a,b) -> Fmt.fprintf out "(@[%a @<1>×@ %a@])" pp_ty a pp_ty b
  | Sum (a,b) -> Fmt.fprintf out "(@[%a +@ %a@])" pp_ty a pp_ty b
;;
#install_printer pp_ty;;

#logic;;

val pp_ty : Fmt.t -> ty -> unit = <fun>


In [27]:

(** Simplicity expressions *)
type expr =
  | Iden
  | Unit
  | Inj_l of expr
  | Inj_r of expr
  | Comp of ty * expr * expr  (* intermediate type needs to be provided *)
  | Case of expr * expr
  | Pair of expr * expr
  | Take of expr
  | Drop of expr
  
(* constructors *)

let iden = Iden [@@macro]
let unit = Unit [@@macro]
let inj_l x = Inj_l x [@@macro]
let inj_r x = Inj_r x [@@macro]
let comp ty a b = Comp(ty,a,b) [@@macro]
let case a b = Case(a,b) [@@macro]
let pair a b = Pair(a,b) [@@macro]
let take s = Take s [@@macro]
let drop s = Drop s [@@macro]

type expr =
    Iden
  | Unit
  | Inj_l of expr
  | Inj_r of expr
  | Comp of ty * expr * expr
  | Case of expr * expr
  | Pair of expr * expr
  | Take of expr
  | Drop of expr
val iden : expr = Iden
val unit : expr = Unit
val inj_l : expr -> expr = <fun>
val inj_r : expr -> expr = <fun>
val comp : ty -> expr -> expr -> expr = <fun>
val case : expr -> expr -> expr = <fun>
val pair : expr -> expr -> expr = <fun>
val take : expr -> expr = <fun>
val drop : expr -> expr = <fun>


In [51]:
let rec pp_expr out = function
  | Iden -> Fmt.string out "iden"
  | Unit -> Fmt.string out "unit"
  | Comp (_,a,b) -> Fmt.fprintf out "(@[comp@ %a@ %a@])" pp_expr a pp_expr b
  | Inj_l a -> Fmt.fprintf out "(@[inj_l@ %a@])" pp_expr a
  | Inj_r a -> Fmt.fprintf out "(@[inj_r@ %a@])" pp_expr a
  | Take a -> Fmt.fprintf out "(@[take@ %a@])" pp_expr a
  | Drop a -> Fmt.fprintf out "(@[drop@ %a@])" pp_expr a
  | Case (a,b) -> Fmt.fprintf out "(@[<hv1>case@ %a@ %a@])" pp_expr a pp_expr b
  | Pair (a,b) -> Fmt.fprintf out "(@[<hv1>pair@ %a@ %a@])" pp_expr a pp_expr b
[@@program]
;;

#install_printer pp_expr;;

val pp_expr : Fmt.t -> expr -> unit = <fun>


## Typing

We could not write a mere typechecking function with the paper expressions, because the typing rule for `Comp` is a "cut". Instead, we ask the user to provide this middle type in `Comp` explicitly.

In [29]:

(** Predicate: [expr_ty e ty = true] means that [e : ty] holds *)
let rec expr_ty (e:expr) (ty:top_ty) : bool =
  match e, ty with
  | Iden, (a, b) -> a = b
  | Unit, (_, One) -> true
  | Comp (b,s,t), (a, c) ->
    expr_ty s (a,b) && expr_ty t (b,c)
  | Case (s,t), (Prod(Sum (a,b),c), d) ->
    expr_ty s (prod a c, d) && expr_ty t (prod b c, d)
  | Pair(s,t), (a,Prod(b,c)) ->
    expr_ty s (a, b) && expr_ty t (a, c)
  | Take t, (Prod(a,b),c) -> expr_ty t (a,c)
  | Drop t, (Prod(a,b),c) -> expr_ty t (b,c)
  | Inj_l t, (a, Sum(b,c)) -> expr_ty t (a,b)
  | Inj_r t, (a, Sum(b,c)) -> expr_ty t (a,c)
  | Unit, _ | Case _, _ | Take _, _ | Drop _, _ 
  | Pair _, _ | Inj_l _, _ | Inj_r _, _ 
    -> false
;;



val expr_ty : expr -> top_ty -> bool = <fun>


termination proof Termination proof call (expr_ty Destruct(Comp, 1, e) (ty.0, Destruct(Comp, 0, e))) from (expr_ty e ty) original (expr_ty e ty) sub (expr_ty Destruct(Comp, 1, e) (ty.0, Destruct(Comp, 0, e))) original ordinal (Ordinal.Int((Ordinal.count e))) sub ordinal (Ordinal.Int((Ordinal.count Destruct(Comp, 1, e)))) path [Is_a(Comp, e) && (not ((ty.1 = One) && (e = Unit))) && (not (e = Iden))] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.011s details Expand Expand start[0.011s]
((Is_a(Comp, e) &&
 ((not ((ty.1 = One) && (e = Unit))) &&
 ((not (e = Iden)) &&
 (((Ordinal.count e) >= 0) && ((Ordinal.count Destruct(Comp, 1, e)) >= 0)))))
 ==>
 (((not
 (Is_a(Comp, Destruct(Comp, 1, e)) &&
 ((not
 (((ty.0, Destruct(Comp, 0, e)).1 = One) &&
 (Destruct(Comp, 1, e) = Unit)))
 && (not (Destruct(Comp, 1, e) = Iden)))))
 &&
 ((not
 (Is_a(Comp, Destruct(Comp, 1, e)) &&
 ((not
 (((ty.0, Destruct(Comp, 0, e)).1 = One) &&
 (Destruct(Comp, 1, e) = Unit)))
 && (not (Destruct(Comp, 1, e) = Iden)))))
 &&
 ((not
 ((Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).0) &&
 (Is_a(Sum, Destruct(Prod, 0, (ty.0, Destruct(Comp, 0, e)).0)) 
 && Is_a(Case, Destruct(Comp, 1, e))))
 &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 ((not
 (((ty.0, Destruct(Comp, 0, e)).1 = One) &&
 (Destruct(Comp, 1, e) = Unit)))
 && (not (Destruct(Comp, 1, e) = Iden))))))
 &&
 ((not
 ((Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).0) &&
 (Is_a(Sum, Destruct(Prod, 0, (ty.0, Destruct(Comp, 0, e)).0)) 
 && Is_a(Case, Destruct(Comp, 1, e))))
 &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 ((not
 (((ty.0, Destruct(Comp, 0, e)).1 = One) &&
 (Destruct(Comp, 1, e) = Unit)))
 && (not (Destruct(Comp, 1, e) = Iden))))))
 &&
 ((not
 ((Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).1) &&
 Is_a(Pair, Destruct(Comp, 1, e)))
 &&
 ((not
 (Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).0) &&
 (Is_a(Sum, Destruct(Prod, 0, (ty.0, Destruct(Comp, 0, e)).0)) 
 && Is_a(Case, Destruct(Comp, 1, e)))))
 &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 ((not
 (((ty.0, Destruct(Comp, 0, e)).1 = One) &&
 (Destruct(Comp, 1, e) = Unit)))
 && (not (Destruct(Comp, 1, e) = Iden)))))))
 &&
 ((not
 ((Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).1) &&
 Is_a(Pair, Destruct(Comp, 1, e)))
 &&
 ((not
 (Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).0) &&
 (Is_a(Sum, Destruct(Prod, 0, (ty.0, Destruct(Comp, 0, e)).0)) 
 && Is_a(Case, Destruct(Comp, 1, e)))))
 &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 ((not
 (((ty.0, Destruct(Comp, 0, e)).1 = One) &&
 (Destruct(Comp, 1, e) = Unit)))
 && (not (Destruct(Comp, 1, e) = Iden)))))))
 &&
 ((not
 ((Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).0) &&
 Is_a(Take, Destruct(Comp, 1, e)))
 &&
 ((not
 (Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).1) &&
 Is_a(Pair, Destruct(Comp, 1, e))))
 &&
 ((not
 (Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).0) &&
 (Is_a(Sum, Destruct(Prod, 0, (ty.0, Destruct(Comp, 0, e)).0))
 && Is_a(Case, Destruct(Comp, 1, e)))))
 &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 ((not
 (((ty.0, Destruct(Comp, 0, e)).1 = One) &&
 (Destruct(Comp, 1, e) = Unit)))
 && (not (Destruct(Comp, 1, e) = Iden))))))))
 &&
 ((not
 ((Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).0) &&
 Is_a(Drop, Destruct(Comp, 1, e)))
 &&
 ((not
 (Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).0) &&
 Is_a(Take, Destruct(Comp, 1, e))))
 &&
 ((not
 (Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).1) &&
 Is_a(Pair, Destruct(Comp, 1, e))))
 &&
 ((not
 (Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).0) &&
 (Is_a(Sum,
 Destruct(Prod, 0, (ty.0, Destruct(Comp, 0, e)).0)) 
 && Is_a(Case, Destruct(Comp, 1, e)))))
 &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 ((not
 (((ty.0, Destruct(Comp, 0, e)).1 = One) &&
 (Destruct(Comp, 1, e) = Unit)))
 && (not (Destruct(Comp, 1, e) = Iden)))))))))
 &&
 ((not
 ((Is_a(Sum, (ty.0, Destruct(Comp, 0, e)).1) &&
 Is_a(Inj_l, Destruct(Comp, 1, e)))
 &&
 ((not
 (Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).0) &&
 Is_a(Drop, Destruct(Comp, 1, e))))
 &&
 ((not
 (Is_a(Prod, (ty.0, Destruct(Comp, 0, e)).0

We might think that typing of an `expr` is unique, but it is not the case:

In [30]:
#timeout 5;;

verify
  (fun e ty1 ty2 ->
     expr_ty e ty1 && expr_ty e ty2 ==> ty1 = ty2)


- : expr -> top_ty -> top_ty -> bool = <fun>
2 base ground recursive instances
Counterexample (after 2 steps, 0.012s):
 let e = Iden
 let ty1 = (One, One)
 let ty2 = (Prod (Prod (One, One), One), Prod (Prod (One, One), One))
module CX : sig val e : expr val ty1 : ty * ty val ty2 : ty * ty end
[✗] Conjecture refuted.


Refuted proof summary full graph ground_instances 2 definitions 2 inductions 0 search_time 0.013s details Expand smt_stats (:added-eqs 458
 :conflicts 28
 :datatype-accessor-ax 50
 :datatype-constructor-ax 108
 :datatype-occurs-check 1498
 :datatype-splits 164
 :decisions 267
 :del-clause 2
 :final-checks 18
 :max-memory 40.49
 :memory 34.23
 :minimized-lits 3
 :mk-bool-var 468
 :mk-clause 172
 :num-allocs 7312198413.00
 :num-checks 5
 :propagations 198
 :rlimit-count 6161226) Expand start[0.013s]
(((expr_ty :var_0: :var_1:) && (expr_ty :var_0: :var_2:)) ==>
 (:var_1: = :var_2:)) pre-process ((not ((expr_ty :var_0: :var_1:) && (expr_ty :var_0: :var_2:))) ||
 (:var_1: = :var_2:)) unroll expr [(expr_ty_2042 e_2134 ty2_2136)] expansions prod unroll expr [(expr_ty_2042 e_2134 ty1_2135)] expansions prod Sat (Some let e = Iden
 let ty1 = (One, One)
 let ty2 = (Prod (Prod (One, One), One), Prod (Prod (One, One), One))) let e = Iden
 let ty1 = (One, One)
 let ty2 = (Prod (Prod (One, One), One), Prod (Prod (One, One), One))

In [47]:
#timeout 3;;
#unroll 50;;

verify
  (fun e ty1 ty2 ->
     e <> Unit && e <> Iden ==>
     expr_ty e ty1 && expr_ty e ty2 ==> ty1 = ty2)


- : expr -> top_ty -> top_ty -> bool = <fun>
2 base ground recursive instances
Counterexample (after 5 steps, 0.121s):
 let e = Inj_r (Iden)
 let ty1 = (Prod (Prod (Prod (One, One), Prod (One, One)), One), Sum (One, Prod (Prod (Prod (One, One), Prod (One, One)), One)))
 let ty2 = (Prod (Sum (Sum (Sum (One, One), Sum (One, One)), Sum (Sum (One, One), Sum (One, One))), One), Sum (One, Prod (Sum (Sum (Sum (One, One), Sum (One, One)), Sum (Sum (One, One), Sum (One, One))), One)))
module CX : sig val e : expr val ty1 : ty * ty val ty2 : ty * ty end
[✗] Conjecture refuted.


Refuted proof summary full graph ground_instances 5 definitions 5 inductions 0 search_time 0.123s details Expand smt_stats (:added-eqs 31653
 :conflicts 300
 :datatype-accessor-ax 1528
 :datatype-constructor-ax 4981
 :datatype-occurs-check 130215
 :datatype-splits 23481
 :decisions 14367
 :del-clause 14
 :final-checks 276
 :max-memory 44.74
 :memory 33.58
 :minimized-lits 4
 :mk-bool-var 19211
 :mk-clause 656
 :num-allocs 15624884894.00
 :num-checks 11
 :propagations 3725
 :restarts 1
 :rlimit-count 12243415) Expand start[0.123s]
(((not (:var_0: = Unit)) && (not (:var_0: = Iden))) ==>
 (((expr_ty :var_0: :var_1:) && (expr_ty :var_0: :var_2:)) ==>
 (:var_1: = :var_2:))) pre-process (((not ((not (:var_0: = Unit)) && (not (:var_0: = Iden)))) ||
 (not ((expr_ty :var_0: :var_1:) && (expr_ty :var_0: :var_2:))))
 || (:var_1: = :var_2:)) unroll expr [(expr_ty_2042 e_2304 ty1_2305)] expansions prod unroll expr [(expr_ty_2042 e_2304 ty2_2306)] expansions prod unroll expr [(expr_ty_2042 (Comp_2006_ARG_1_2014 e_2304)
 (|tuple_mk.`(ty * ty)`_1997_1998|
 (|tuple_get.`(ty * ty)`_1997.0_1999| ty1_2305)
 (Comp_2006_ARG_0_2013 e_2304)))] expansions prod unroll expr [(expr_ty_2042 (Inj_r_2005_ARG_0_2012 e_2304)
 (|tuple_mk.`(ty * ty)`_1997_1998|
 (|tuple_get.`(ty * ty)`_1997.0_1999| ty2_2306)
 (Sum_1984_ARG_1_1988 (|tuple_get.`(ty * ty)`_1997.1_2000| ty2_2306))))] expansions prod unroll expr [(expr_ty_2042 (Inj_r_2005_ARG_0_2012 e_2304)
 (|tuple_mk.`(ty * ty)`_1997_1998|
 (|tuple_get.`(ty * ty)`_1997.0_1999| ty1_2305)
 (Sum_1984_ARG_1_1988 (|tuple_get.`(ty * ty)`_1997.1_2000| ty1_2305))))] expansions prod Sat (Some let e = Inj_r (Iden)
 let ty1 = (Prod (Prod (Prod (One, One), Prod (One, One)), One), Sum (One, Prod (Prod (Prod (One, One), Prod (One, One)), One)))
 let ty2 = (Prod (Sum (Sum (Sum (One, One), Sum (One, One)), Sum (Sum (One, One), Sum (One, One))), One), Sum (One, Prod (Sum (Sum (Sum (One, One), Sum (One, One)), Sum (Sum (One, One), Sum (One, One))), One)))) let e = Inj_r (Iden)
 let ty1 = (Prod (Prod (Prod (One, One), Prod (One, One)), One), Sum (One, Prod (Prod (Prod (One, One), Prod (One, One)), One)))
 let ty2 = (Prod (Sum (Sum (Sum (One, One), Sum (One, One)), Sum (Sum (One, One), Sum (One, One))), One), Sum (One, Prod (Sum (Sum (Sum (One, One), Sum (One, One)), Sum (Sum (One, One), Sum (One, One))), One)))

In [52]:
(* check that the counter-example is indeed valid *)
expr_ty CX.e CX.ty1;;
expr_ty CX.e CX.ty2;;
CX.ty1 = CX.ty2;;

CX.e;;

- : bool = true
- : bool = true
- : bool = false
- : expr = (inj_r iden)


## Evaluation


Evaluation reduces a term into a value. Values, like expressions, are typed.

In [32]:
(* first, we need values *)

(** Values obtained when evaluating Simplicity expressions *)
type value =
    | V_unit
    | V_left of value
    | V_right of value
    | V_pair of value * value
    
let v_unit = V_unit [@@macro]
let v_left x = V_left x [@@macro]
let v_right x = V_right x [@@macro]
let v_pair x y = V_pair (x,y) [@@macro]


type value =
    V_unit
  | V_left of value
  | V_right of value
  | V_pair of value * value
val v_unit : value = V_unit
val v_left : value -> value = <fun>
val v_right : value -> value = <fun>
val v_pair : value -> value -> value = <fun>


In [53]:
let rec pp_value out = function
  | V_unit -> Format.fprintf out "()"
  | V_left v -> Format.fprintf out "@<1>σ0%a" pp_value v
  | V_right v -> Format.fprintf out "@<1>σ1%a" pp_value v
  | V_pair (v1,v2) -> Format.fprintf out "(@[%a,@ %a@])" pp_value v1 pp_value v2
[@@program]
;;

#install_printer pp_value ;;

val pp_value : Format.formatter -> value -> unit = <fun>


In [54]:
(** Typing predicate for values. [value_ty v ty] is true iff [v : ty] *)
let rec value_ty (v:value) (ty:ty) : bool =
  match v, ty with
  | V_unit, One -> true
  | V_pair (v1,v2), Prod(a,b) -> value_ty v1 a && value_ty v2 b
  | V_left v', Sum (a,_) -> value_ty v' a
  | V_right v', Sum (_,b) -> value_ty v' b
  | V_unit, _ | V_pair _, _ | V_left _, _ | V_right _, _ -> false


val value_ty : value -> ty -> bool = <fun>


termination proof Termination proof call (value_ty Destruct(V_pair, 0, v) Destruct(Prod, 0, ty)) from (value_ty v ty) original (value_ty v ty) sub (value_ty Destruct(V_pair, 0, v) Destruct(Prod, 0, ty)) original ordinal (Ordinal.Int((Ordinal.count v))) sub ordinal (Ordinal.Int((Ordinal.count Destruct(V_pair, 0, v)))) path [(Is_a(Prod, ty) && Is_a(V_pair, v)) && (not ((ty = One) && (v = V_unit)))] proof detailed proof summary full ground_instances 2 definitions 0 inductions 0 search_time 0.006s details Expand Expand start[0.006s]
(((Is_a(Prod, ty) && Is_a(V_pair, v)) &&
 ((not ((ty = One) && (v = V_unit))) &&
 (((Ordinal.count v) >= 0) && ((Ordinal.count Destruct(V_pair, 0, v)) >= 0))))
 ==>
 (((not
 ((Is_a(Prod, Destruct(Prod, 0, ty)) &&
 Is_a(V_pair, Destruct(V_pair, 0, v)))
 &&
 (not
 ((Destruct(Prod, 0, ty) = One) && (Destruct(V_pair, 0, v) = V_unit)))))
 &&
 ((not
 ((Is_a(Prod, Destruct(Prod, 0, ty)) &&
 Is_a(V_pair, Destruct(V_pair, 0, v)))
 &&
 (not
 ((Destruct(Prod, 0, ty) = One) && (Destruct(V_pair, 0, v) = V_unit)))))
 &&
 ((not
 ((Is_a(Sum, Destruct(Prod, 0, ty)) &&
 Is_a(V_left, Destruct(V_pair, 0, v)))
 &&
 ((not
 (Is_a(Prod, Destruct(Prod, 0, ty)) &&
 Is_a(V_pair, Destruct(V_pair, 0, v))))
 &&
 (not
 ((Destruct(Prod, 0, ty) = One) && (Destruct(V_pair, 0, v) = V_unit))))))
 &&
 (not
 ((Is_a(Sum, Destruct(Prod, 0, ty)) &&
 Is_a(V_right, Destruct(V_pair, 0, v)))
 &&
 ((not
 (Is_a(Sum, Destruct(Prod, 0, ty)) &&
 Is_a(V_left, Destruct(V_pair, 0, v))))
 &&
 ((not
 (Is_a(Prod, Destruct(Prod, 0, ty)) &&
 Is_a(V_pair, Destruct(V_pair, 0, v))))
 &&
 (not
 ((Destruct(Prod, 0, ty) = One) && (Destruct(V_pair, 0, v) = V_unit))))))))))
 ||
 ((Ordinal.Int((Ordinal.count Destruct(V_pair, 0, v)))) <<
 (Ordinal.Int((Ordinal.count v)))))) pre-process (((not
 ((((Is_a(Prod, ty) && Is_a(V_pair, v)) &&
 (not ((ty = One) && (v = V_unit))))
 && ((Ordinal.count v) >= 0))
 && ((Ordinal.count Destruct(V_pair, 0, v)) >= 0)))
 ||
 (((not
 ((Is_a(Prod, Destruct(Prod, 0, ty)) &&
 Is_a(V_pair, Destruct(V_pair, 0, v)))
 &&
 (not
 ((Destruct(Prod, 0, ty) = One) && (Destruct(V_pair, 0, v) = V_unit)))))
 &&
 (not
 (((Is_a(Sum, Destruct(Prod, 0, ty)) &&
 Is_a(V_left, Destruct(V_pair, 0, v)))
 &&
 (not
 (Is_a(Prod, Destruct(Prod, 0, ty)) &&
 Is_a(V_pair, Destruct(V_pair, 0, v)))))
 &&
 (not
 ((Destruct(Prod, 0, ty) = One) && (Destruct(V_pair, 0, v) = V_unit))))))
 &&
 (not
 ((((Is_a(Sum, Destruct(Prod, 0, ty)) &&
 Is_a(V_right, Destruct(V_pair, 0, v)))
 &&
 (not
 (Is_a(Sum, Destruct(Prod, 0, ty)) &&
 Is_a(V_left, Destruct(V_pair, 0, v)))))
 &&
 (not
 (Is_a(Prod, Destruct(Prod, 0, ty)) &&
 Is_a(V_pair, Destruct(V_pair, 0, v)))))
 &&
 (not
 ((Destruct(Prod, 0, ty) = One) && (Destruct(V_pair, 0, v) = V_unit)))))))
 ||
 ((Ordinal.Int((Ordinal.count Destruct(V_pair, 0, v)))) <<
 (Ordinal.Int((Ordinal.count v))))) unroll expr [(Ordinal.<<_128
 (Ordinal.Int_119 (count_value_2340 (V_pair_2156_ARG_0_2159 v_2335)))
 (Ordinal.Int_119 (count_value_2340 v_2335)))] expansions unroll expr [(count_value_2340 v_2335)] expansions call (value_ty Destruct(V_pair, 1, v) Destruct(Prod, 1, ty)) from (value_ty v ty) original (value_ty v ty) sub (value_ty Destruct(V_pair, 1, v) Destruct(Prod, 1, ty)) original ordinal (Ordinal.Int((Ordinal.count v))) sub ordinal (Ordinal.Int((Ordinal.count Destruct(V_pair, 1, v)))) path [(Is_a(Prod, ty) && Is_a(V_pair, v)) && (not ((ty = One) && (v = V_unit)))] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.011s details Expand Expand start[0.011s]
(((Is_a(Prod, ty) && Is_a(V_pair, v)) &&
 ((not ((ty = One) && (v = V_unit))) &&
 (((Ordinal.count v) >= 0) && ((Ordinal.count Destruct(V_pair, 1, v)) >= 0))))
 ==>
 (((not
 ((Is_a(Prod, Destruct(Prod, 1, ty)) &&
 Is_a(V_pair, Destruct(V_pair, 1, v)))
 &&
 (not
 ((Destruct(Prod, 1, ty) = One) && (Destruct(V_pair, 1, v) = V_unit)))))
 &&
 ((not
 ((Is_a(Prod, Destruct(Prod, 1, ty)) &&
 Is_a(V_pair, Destruct(V_pair, 1, v)))
 &&
 (not
 ((Destruct(Prod, 1, ty)

Evaluation takes an expression of type $ A \vdash B $, and a value of type $A$, and
returns a value of type $B$.

It always terminates and takes predictable time and space.

In [55]:

(** Main evaluation function. It is partial because of type errors *)
let rec eval (e:expr) (v:value) : value option =
    let open Option in
    match e, v with
    | Unit, _ -> Some V_unit
    | Comp (_,s,t), _ ->
      eval s v >>= fun v' -> eval t v'
    | Iden, _ -> Some v
    | Inj_l t, _ -> eval t v >|= v_left
    | Inj_r t, _ -> eval t v >|= v_right
    | Case (s, t), V_pair (V_left a, c) ->
      eval s (v_pair a c)
    | Case (s, t), V_pair (V_right b, c) ->
      eval t (v_pair b c)
    | Pair (s, t), _ ->
      eval s v >>= fun v1 ->
      eval t v >|= fun v2 ->
      v_pair v1 v2 
    | Take t, V_pair (a,_) ->
      eval t a 
    | Drop t, V_pair (_,b) ->
      eval t b
    | _ ->
      None  (* ill typed *)
[@@adm 0n]

val eval : expr -> value -> value option = <fun>


termination proof Termination proof call (eval Destruct(Comp, 1, e) v) from (eval e v) original (eval e v) sub (eval Destruct(Comp, 1, e) v) original ordinal (Ordinal.Int((Ordinal.count e))) sub ordinal (Ordinal.Int((Ordinal.count Destruct(Comp, 1, e)))) path [Is_a(Comp, e) && (not (e = Unit))] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.017s details Expand Expand start[0.017s]
((Is_a(Comp, e) &&
 ((not (e = Unit)) &&
 (((Ordinal.count e) >= 0) && ((Ordinal.count Destruct(Comp, 1, e)) >= 0))))
 ==>
 (((not
 (Is_a(Comp, Destruct(Comp, 1, e)) && (not (Destruct(Comp, 1, e) = Unit))))
 &&
 ((not
 ((not ((eval Destruct(Comp, 1, Destruct(Comp, 1, e)) v) = None)) 
 &&
 (Is_a(Comp, Destruct(Comp, 1, e)) &&
 (not (Destruct(Comp, 1, e) = Unit)))))
 &&
 ((not
 ((not ((eval Destruct(Comp, 1, Destruct(Comp, 1, e)) v) = None)) 
 &&
 (Is_a(Comp, Destruct(Comp, 1, e)) &&
 (not (Destruct(Comp, 1, e) = Unit)))))
 &&
 ((not
 (Is_a(Inj_l, Destruct(Comp, 1, e)) &&
 ((not (Destruct(Comp, 1, e) = Iden)) &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 (not (Destruct(Comp, 1, e) = Unit))))))
 &&
 ((not
 ((not ((eval Destruct(Inj_l, 0, Destruct(Comp, 1, e)) v) = None)) 
 &&
 (Is_a(Inj_l, Destruct(Comp, 1, e)) &&
 ((not (Destruct(Comp, 1, e) = Iden)) &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 (not (Destruct(Comp, 1, e) = Unit)))))))
 &&
 ((not
 (Is_a(Inj_r, Destruct(Comp, 1, e)) &&
 ((not Is_a(Inj_l, Destruct(Comp, 1, e))) &&
 ((not (Destruct(Comp, 1, e) = Iden)) &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 (not (Destruct(Comp, 1, e) = Unit)))))))
 &&
 ((not
 ((not ((eval Destruct(Inj_r, 0, Destruct(Comp, 1, e)) v) = None))
 &&
 (Is_a(Inj_r, Destruct(Comp, 1, e)) &&
 ((not Is_a(Inj_l, Destruct(Comp, 1, e))) &&
 ((not (Destruct(Comp, 1, e) = Iden)) &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 (not (Destruct(Comp, 1, e) = Unit))))))))
 &&
 ((not
 ((Is_a(V_pair, v) &&
 (Is_a(V_left, Destruct(V_pair, 0, v)) &&
 Is_a(Case, Destruct(Comp, 1, e))))
 &&
 ((not Is_a(Inj_r, Destruct(Comp, 1, e))) &&
 ((not Is_a(Inj_l, Destruct(Comp, 1, e))) &&
 ((not (Destruct(Comp, 1, e) = Iden)) &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 (not (Destruct(Comp, 1, e) = Unit))))))))
 &&
 ((not
 ((Is_a(V_pair, v) &&
 (Is_a(V_right, Destruct(V_pair, 0, v)) &&
 Is_a(Case, Destruct(Comp, 1, e))))
 &&
 ((not
 (Is_a(V_pair, v) &&
 (Is_a(V_left, Destruct(V_pair, 0, v)) &&
 Is_a(Case, Destruct(Comp, 1, e)))))
 &&
 ((not Is_a(Inj_r, Destruct(Comp, 1, e))) &&
 ((not Is_a(Inj_l, Destruct(Comp, 1, e))) &&
 ((not (Destruct(Comp, 1, e) = Iden)) &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 (not (Destruct(Comp, 1, e) = Unit)))))))))
 &&
 ((not
 (Is_a(Pair, Destruct(Comp, 1, e)) &&
 ((not
 (Is_a(V_pair, v) &&
 (Is_a(V_right, Destruct(V_pair, 0, v)) &&
 Is_a(Case, Destruct(Comp, 1, e)))))
 &&
 ((not
 (Is_a(V_pair, v) &&
 (Is_a(V_left, Destruct(V_pair, 0, v)) &&
 Is_a(Case, Destruct(Comp, 1, e)))))
 &&
 ((not Is_a(Inj_r, Destruct(Comp, 1, e))) &&
 ((not Is_a(Inj_l, Destruct(Comp, 1, e))) &&
 ((not (Destruct(Comp, 1, e) = Iden)) &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 (not (Destruct(Comp, 1, e) = Unit))))))))))
 &&
 ((not
 ((not ((eval Destruct(Pair, 0, Destruct(Comp, 1, e)) v) = None))
 &&
 (Is_a(Pair, Destruct(Comp, 1, e)) &&
 ((not
 (Is_a(V_pair, v) &&
 (Is_a(V_right, Destruct(V_pair, 0, v)) &&
 Is_a(Case, Destruct(Comp, 1, e)))))
 &&
 ((not
 (Is_a(V_pair, v) &&
 (Is_a(V_left, Destruct(V_pair, 0, v)) &&
 Is_a(Case, Destruct(Comp, 1, e)))))
 &&
 ((not Is_a(Inj_r, Destruct(Comp, 1, e))) &&
 ((not Is_a(Inj_l, Destruct(Comp, 1, e))) &&
 ((not (Destruct(Comp, 1, e) = Iden)) &&
 ((not Is_a(Comp, Destruct(Comp, 1, e))) &&
 (not (Destruct(Comp, 1, e) = Unit)))))))))))
 &&
 ((not
 ((not
 ((eval Destruct(Pair, 1, Destruct(Comp, 1, e)) v) = None))
 &&
 ((not
 ((eval Destruct(Pair, 0, Destruct(Comp, 1, e)) v) = None))
 &&
 (Is_a(Pair, Destruct(Comp, 1, e)) &&
 ((not
 (Is_a(V_pair, v) &&
 (Is_a(V_right, Destruct(V_pair, 0, v)) &&
 Is_a

### Example

We can define the type of individual bits: $\mathbb{2} \equiv \mathbb{1} + \mathbb{1}$

In [36]:
let bit : ty = sum one one

let b_zero = v_left v_unit
let b_one  = v_right v_unit
;;

value_ty b_zero bit ;;
value_ty b_one  bit ;;

val bit : ty = (1 + 1)
val b_zero : value = σ0()
val b_one : value = σ1()
- : bool = true
- : bool = true


In [56]:
(* bit negation *)

let bit_not : expr =
    comp (prod bit one) (pair iden unit) (case (inj_r unit) (inj_l unit))
;;

b_zero, eval bit_not b_zero ;;
b_one,  eval bit_not b_one ;;


expr_ty bit_not (bit, bit) ;;

val bit_not : expr = (comp (pair iden unit) (case (inj_r unit) (inj_l unit)))
- : value * value option = (σ0(), Some σ1())
- : value * value option = (σ1(), Some σ0())
- : bool = true


In [57]:

(* how to infer the intermediate type of `bit_not` *)

verify (fun ty ->
  let bit_not = comp ty (pair iden unit) (case (inj_r unit) (inj_l unit)) in
  not (expr_ty bit_not (bit,bit)))
;;
  
CX.ty;;

(* check result *)
let bit_not' =
  comp CX.ty (pair iden unit) (case (inj_r unit) (inj_l unit))
in
not (expr_ty bit_not' (bit,bit));;

CX.ty = prod bit one;;

- : ty -> bool = <fun>
1 base ground recursive instance
Counterexample (after 11 steps, 0.027s):
 let ty = Prod (Sum (One, One), One)
module CX : sig val ty : ty end
[✗] Conjecture refuted.
- : ty = ((1 + 1) × 1)
- : bool = false
- : bool = true


Refuted proof summary full graph ground_instances 11 definitions 11 inductions 0 search_time 0.028s details Expand smt_stats (:added-eqs 211
 :conflicts 13
 :datatype-accessor-ax 29
 :datatype-constructor-ax 38
 :datatype-occurs-check 1496
 :datatype-splits 17
 :decisions 98
 :del-clause 15
 :final-checks 21
 :max-memory 44.74
 :memory 31.17
 :mk-bool-var 717
 :mk-clause 43
 :num-allocs 28556574319.00
 :num-checks 23
 :propagations 55
 :rlimit-count 12446756) Expand start[0.028s]
(not
 (expr_ty Comp(:var_0:, Pair(Iden, Unit), Case((Inj_r(Unit)), (Inj_l(Unit))))
 (Sum(One, One), Sum(One, One)))) unroll expr [(expr_ty_2042
 (Comp_2006 ty_2461 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982)))] expansions prod unroll expr [(let
 ((a!1
 (Comp_2006 ty_2461 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))
 (let
 ((a!2
 (|tuple_mk.`(ty * ty)`_1997_1998| (Comp_2006_ARG_0_2013 a!1)
 (|tuple_get.`(ty * ty)`_1997.1_2000|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982))))))
 (expr_ty_2042 (Comp_2006_ARG_2_2015 a!1) a!2)))] expansions prod unroll expr [(let
 ((a!1
 (Comp_2006 ty_2461 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))
 (let
 ((a!2
 (|tuple_mk.`(ty * ty)`_1997_1998|
 (|tuple_get.`(ty * ty)`_1997.0_1999|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982)))
 (Comp_2006_ARG_0_2013 a!1))))
 (expr_ty_2042 (Comp_2006_ARG_1_2014 a!1) a!2)))] expansions prod unroll expr [(let
 ((a!1
 (Comp_2006 ty_2461 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))
 (let
 ((a!2
 (|tuple_mk.`(ty * ty)`_1997_1998| (Comp_2006_ARG_0_2013 a!1)
 (|tuple_get.`(ty * ty)`_1997.1_2000|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982))))))
 (let
 ((a!3
 (Prod_1983
 (Sum_1984_ARG_1_1988
 (Prod_1983_ARG_0_1985 (|tuple_get.`(ty * ty)`_1997.0_1999| a!2)))
 (Prod_1983_ARG_1_1986 (|tuple_get.`(ty * ty)`_1997.0_1999| a!2)))))
 (expr_ty_2042 (Case_2007_ARG_1_2017 (Comp_2006_ARG_2_2015 a!1))
 (|tuple_mk.`(ty * ty)`_1997_1998| a!3
 (|tuple_get.`(ty * ty)`_1997.1_2000| a!2))))))] expansions prod unroll expr [(let
 ((a!1
 (Inj_r_2005_ARG_0_2012
 (Comp_2006 ty_2461 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))
 (a!2
 (Sum_1984_ARG_1_1988
 (|tuple_get.`(ty * ty)`_1997.1_2000|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982))))))
 (let
 ((a!3
 (|tuple_mk.`(ty * ty)`_1997_1998|
 (|tuple_get.`(ty * ty)`_1997.0_1999|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982)))
 a!2)))
 (expr_ty_2042 a!1 a!3)))] expansions prod unroll expr [(let
 ((a!1
 (Comp_2006 ty_2461 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))
 (let
 ((a!2
 (|tuple_mk.`(ty * ty)`_1997_1998| (Comp_2006_ARG_0_2013 a!1)
 (|tuple_get.`(ty * ty)`_1997.1_2000|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982))))))
 (let
 ((a!3
 (Prod_1983
 (Sum_1984_ARG_0_1987
 (Prod_1983_ARG_0_1985 (|tuple_get.`(ty * ty)`_1997.0_1999| a!2)))
 (Prod_1983_ARG_1_1986 (|tuple_get.`(ty * ty)`_1997.0_1999| a!2)))))
 (expr_ty_2042 (Case_2007_ARG_0_2016 (Comp_2006_ARG_2_2015 a!1))
 (|tuple_mk.`(ty * ty)`_1997_1998| a!3
 (|tuple_get.`(ty * ty)`_1997.1_2000| a!2))))))] expansions prod unroll expr [(let
 ((a!1
 (Comp_2006 ty_2461 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))
 (let
 ((a!2
 (|tuple_mk.`(ty * ty)`_1997_1998|
 (|tuple_get.`(ty * ty)`_1997.0_1999|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982)))
 (Comp_2006_ARG_0_2013 

In [41]:
let goal = fun ty ->
  let bit_not = comp ty (pair iden unit) (case (inj_r unit) (inj_l unit)) in
  not (expr_ty bit_not (bit,bit))
;;

Verify.top "goal";;

val goal : ty -> bool = <fun>
1 base ground recursive instance
Counterexample (after 11 steps, 0.020s):
 let ty = Prod (Sum (One, One), One)
module CX : sig val ty : ty end
- : Imandra_lib.Kernel.Verify.t =
Imandra_lib.Kernel.Verify.Refuted
 {Imandra_lib.Kernel.Verify.model = Some <abstr>;
  model_str = " let ty = Prod (Sum (One, One), One)"; proof = Some <abstr>}


Refuted proof summary full graph ground_instances 11 definitions 19 inductions 0 search_time 0.021s details Expand smt_stats (:added-eqs 211
 :conflicts 13
 :datatype-accessor-ax 29
 :datatype-constructor-ax 38
 :datatype-occurs-check 1496
 :datatype-splits 17
 :decisions 98
 :del-clause 15
 :final-checks 21
 :max-memory 44.74
 :memory 18.42
 :mk-bool-var 717
 :mk-clause 43
 :num-allocs 13448466542.00
 :num-checks 23
 :propagations 55
 :rlimit-count 6245722) Expand start[0.021s]
(not
 (expr_ty Comp(:var_0:, Pair(Iden, Unit), Case((Inj_r(Unit)), (Inj_l(Unit))))
 (Sum(One, One), Sum(One, One)))) pre-process (not
 (expr_ty Comp(:var_0:, Pair(Iden, Unit), Case((Inj_r(Unit)), (Inj_l(Unit))))
 (Sum(One, One), Sum(One, One)))) expand [one, one, sum, bit, one, one, sum, bit] unroll expr [(expr_ty_2042
 (Comp_2006 ty_2270 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982)))] expansions prod unroll expr [(let
 ((a!1
 (Comp_2006 ty_2270 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))
 (let
 ((a!2
 (|tuple_mk.`(ty * ty)`_1997_1998| (Comp_2006_ARG_0_2013 a!1)
 (|tuple_get.`(ty * ty)`_1997.1_2000|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982))))))
 (expr_ty_2042 (Comp_2006_ARG_2_2015 a!1) a!2)))] expansions prod unroll expr [(let
 ((a!1
 (Comp_2006 ty_2270 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))
 (let
 ((a!2
 (|tuple_mk.`(ty * ty)`_1997_1998|
 (|tuple_get.`(ty * ty)`_1997.0_1999|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982)))
 (Comp_2006_ARG_0_2013 a!1))))
 (expr_ty_2042 (Comp_2006_ARG_1_2014 a!1) a!2)))] expansions prod unroll expr [(let
 ((a!1
 (Comp_2006 ty_2270 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))
 (let
 ((a!2
 (|tuple_mk.`(ty * ty)`_1997_1998| (Comp_2006_ARG_0_2013 a!1)
 (|tuple_get.`(ty * ty)`_1997.1_2000|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982))))))
 (let
 ((a!3
 (Prod_1983
 (Sum_1984_ARG_1_1988
 (Prod_1983_ARG_0_1985 (|tuple_get.`(ty * ty)`_1997.0_1999| a!2)))
 (Prod_1983_ARG_1_1986 (|tuple_get.`(ty * ty)`_1997.0_1999| a!2)))))
 (expr_ty_2042 (Case_2007_ARG_1_2017 (Comp_2006_ARG_2_2015 a!1))
 (|tuple_mk.`(ty * ty)`_1997_1998| a!3
 (|tuple_get.`(ty * ty)`_1997.1_2000| a!2))))))] expansions prod unroll expr [(let
 ((a!1
 (Inj_r_2005_ARG_0_2012
 (Comp_2006 ty_2270 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))
 (a!2
 (Sum_1984_ARG_1_1988
 (|tuple_get.`(ty * ty)`_1997.1_2000|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982))))))
 (let
 ((a!3
 (|tuple_mk.`(ty * ty)`_1997_1998|
 (|tuple_get.`(ty * ty)`_1997.0_1999|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982)))
 a!2)))
 (expr_ty_2042 a!1 a!3)))] expansions prod unroll expr [(let
 ((a!1
 (Comp_2006 ty_2270 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))
 (let
 ((a!2
 (|tuple_mk.`(ty * ty)`_1997_1998| (Comp_2006_ARG_0_2013 a!1)
 (|tuple_get.`(ty * ty)`_1997.1_2000|
 (|tuple_mk.`(ty * ty)`_1997_1998| (Sum_1984 One_1982 One_1982)
 (Sum_1984 One_1982 One_1982))))))
 (let
 ((a!3
 (Prod_1983
 (Sum_1984_ARG_0_1987
 (Prod_1983_ARG_0_1985 (|tuple_get.`(ty * ty)`_1997.0_1999| a!2)))
 (Prod_1983_ARG_1_1986 (|tuple_get.`(ty * ty)`_1997.0_1999| a!2)))))
 (expr_ty_2042 (Case_2007_ARG_0_2016 (Comp_2006_ARG_2_2015 a!1))
 (|tuple_mk.`(ty * ty)`_1997_1998| a!3
 (|tuple_get.`(ty * ty)`_1997.1_2000| a!2))))))] expansions prod unroll expr [(let
 ((a!1
 (Comp_2006 ty_2270 (Pair_2008 Iden_2002 Unit_2003)
 (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))
 (let
 ((a!2
 (|tuple_mk.`(ty

## Well Typed Expressions

Any well typed expressions evaluates to a value successfully.

In [42]:

theorem eval_well_typed (e:expr) (a, b : ty * ty) (v:value) =
  expr_ty e (a,b) &&
  value_ty v a
  ==>
  begin match eval e v with
    | None -> false
    | Some v' -> value_ty v' b
  end
[@@induct e]
;;


val eval_well_typed : expr -> ty * ty -> value -> bool = <fun>
Goal:

let (a : ty) = _x.0 and (b : ty) = _x.1 in
(((expr_ty e (a, b)) && (value_ty v a)) ==>
 if ((eval e v) = None) then false
 else let (v' : value) = Option.get((eval e v)) in (value_ty v' b)).

File "_none_", line 1:
Error: Invalid induction: (e : expr) is not a list


### Example: half adder

In [59]:
let two_two = prod bit bit 


let half_adder =
  case (drop (pair (inj_l unit) iden))
       (drop (pair iden bit_not))


val two_two : ty = ((1 + 1) × (1 + 1))
val half_adder : expr =
  (case
    (drop (pair (inj_l unit) iden))
    (drop
     (pair iden (comp (pair iden unit) (case (inj_r unit) (inj_l unit))))))


In [60]:
expr_ty half_adder (prod bit bit, two_two);;

- : bool = true


In [62]:
(* TODO: use `instances` to find the type of `half_adder` *)
#unroll 100;;

verify (fun ty -> not @@ expr_ty half_adder ty);;

- : top_ty -> bool = <fun>
1 base ground recursive instance
[?] Unknown.


ground_instances,22
definitions,22
inductions,0
search_time,3.657s
details,Expand
expr,[(let ((a!1 (Pair_2008 Iden_2002 (Comp_2006 (Prod_1983 (Sum_1984 One_1982 One_1982) One_1982) (Pair_2008 Iden_2002 Unit_2003) (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003)))))) (let ((a!2 (Case_2007 (Drop_2010 (Pair_2008 (Inj_l_2004 Unit_2003) Iden_2002)) (Drop_2010 a!1)))) (expr_ty_2042 a!2 ty_2482)))]
expansions,prod
expr,[(let ((a!1 (Pair_2008 Iden_2002 (Comp_2006 (Prod_1983 (Sum_1984 One_1982 One_1982) One_1982) (Pair_2008 Iden_2002 Unit_2003) (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003))))) (a!3 (Prod_1983 (Sum_1984_ARG_1_1988 (Prod_1983_ARG_0_1985 (|tuple_get.`(ty * ty)`_1997.0_1999| ty_2482))) (Prod_1983_ARG_1_1986 (|tuple_get.`(ty * ty)`_1997.0_1999| ty_2482))))) (let ((a!2 (Case_2007 (Drop_2010 (Pair_2008 (Inj_l_2004 Unit_2003) Iden_2002)) (Drop_2010 a!1)))) (expr_ty_2042 (Case_2007_ARG_1_2017 a!2) (|tuple_mk.`(ty * ty)`_1997_1998| a!3 (|tuple_get.`(ty * ty)`_1997.1_2000| ty_2482)))))]
expansions,prod
expr,[(let ((a!1 (Pair_2008 Iden_2002 (Comp_2006 (Prod_1983 (Sum_1984 One_1982 One_1982) One_1982) (Pair_2008 Iden_2002 Unit_2003) (Case_2007 (Inj_r_2005 Unit_2003) (Inj_l_2004 Unit_2003))))) (a!3 (Prod_1983 (Sum_1984_ARG_0_1987 (Prod_1983_ARG_0_1985 (|tuple_get.`(ty * ty)`_1997.0_1999| ty_2482))) (Prod_1983_ARG_1_1986 (|tuple_get.`(ty * ty)`_1997.0_1999| ty_2482))))) (let ((a!2 (Case_2007 (Drop_2010 (Pair_2008 (Inj_l_2004 Unit_2003) Iden_2002)) (Drop_2010 a!1)))) (expr_ty_2042 (Case_2007_ARG_0_2016 a!2) (|tuple_mk.`(ty * ty)`_1997_1998| a!3 (|tuple_get.`(ty * ty)`_1997.1_2000| ty_2482)))))]
expansions,prod


In [46]:
#h List.map

doc,Map a function over a list.map f [] = []map f [x] = [f x]map f (x :: tail) = f x :: map f tail
iml,definition(fun (f : 'b -> 'a) -> (fun (l : 'b list) -> if (l = []) then [] else let (x : 'b) = List.hd(l) and (l2 : 'b list) = List.tl(l) in ((f x) :: (List.map f l2))))
name,List.map
signature,('b -> 'a) -> 'b list -> 'a list
recursive,true
call,List.map (f : 'b -> 'a) (l : 'b list)
Measured subset,[l]
parametric,true
location,Characters 12740-12830: #h List.map
